In [2]:
import pandas as pd


def purge():
    import torch
    from numba import cuda
    import gc
    from time import sleep
    try:
        cuda.select_device(0)
        cuda.close()
        cuda.select_device(0)
        gc.collect()
        sleep(2)
        torch.cuda.empty_cache()
    except:
        pass
    
    
checkpoints = [
    "facebook/wav2vec2-large-slavic-voxpopuli-v2",
    # "facebook/wav2vec2-large-960h-lv60-self",
    # "classla/wav2vec2-large-slavic-parlaspeech-hr",
]
optimal_epochs = {
    "facebook/wav2vec2-large-960h-lv60-self": 9,
    "facebook/wav2vec2-large-slavic-voxpopuli-v2": 11,
    "classla/wav2vec2-large-slavic-parlaspeech-hr": 7,  
}
from utils import train_model, eval_model
results = []
for checkpoint in checkpoints:
    train_config = dict(
        model_name_or_path = checkpoint,
        TASK = "emotion_optimal_epochs",
        NUM_EPOCH = optimal_epochs.get(checkpoint),
        output_column = "target",
        input_column = "path",
        data_files = {
            "train": "001_train.csv",
            "validation": "001_dev.csv",
        },
        clip_seconds = 10,
    )

    output_dir = train_model(train_config)
    import numpy as np
    for split in "dev test".split():
        from pathlib import Path
        found_path = str(list(Path(output_dir).glob("checkpoint-*"))[0])
        print(f"For checkpoint: {checkpoint} a path was found: ", found_path)
        eval_config = dict(
            output_column = "target",
            model_name_or_path= found_path ,
            eval_file= f"001_{split}.csv"
        )

        y_true, y_pred = eval_model(eval_config)
        results.append({**eval_config, 
                        **train_config,
                        "split": split,
                        "y_true": y_true, 
                        "y_pred": y_pred})
        content = pd.DataFrame(data=results).to_json(None, 
                                           orient="records",
                                           lines=True,
                                           )
        with open("004_results_new_new.jsonl", "a") as f:
            f.writelines(content)
    import os
    os.system("rm -r models/*")

Using custom data configuration default-ca47a7a6555e75ab
Reusing dataset csv (/home/peterr/.cache/huggingface/datasets/csv/default-ca47a7a6555e75ab/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

A classification problem with 5 classes: ['anger', 'fear', 'happiness', 'neutral', 'sadness']


loading configuration file https://huggingface.co/facebook/wav2vec2-large-slavic-voxpopuli-v2/resolve/main/config.json from cache at /home/peterr/.cache/huggingface/transformers/a9ef28f26feee4cedb0e5104e0b7c3f1ee9b187ec929d279d88c839e96d59f7f.9e2b83ee7b47903bc8babac566bcfe40deb23eecf8488d186749f6b277850a85
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-slavic-voxpopuli-v2",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_

The target sampling rate: 16000


...

For checkpoint: facebook/wav2vec2-large-slavic-voxpopuli-v2 a path was found:  models/facebook_wav2vec2-large-slavic-voxpopuli-v2_emotion_optimal_epochs_/checkpoint-1936


Using custom data configuration default-38cc960bf51fe830
Reusing dataset csv (/home/peterr/.cache/huggingface/datasets/csv/default-38cc960bf51fe830/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

loading configuration file models/facebook_wav2vec2-large-slavic-voxpopuli-v2_emotion_optimal_epochs_/checkpoint-1936/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "models/facebook_wav2vec2-large-slavic-voxpopuli-v2_emotion_optimal_epochs_/checkpoint-1936",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForSpeechClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_to

  0%|          | 0/51 [00:00<?, ?ba/s]